
# Generative AI integration (Phase 4)



In this phase, we aimed to enhance our system's interactivity and overall user experience by integrating Generative AI capabilities, specifically leveraging a model like GPT. The main objective was to move beyond traditional classification outputs and provide responses that are personalized, informative, and human-like—tailored to each user's unique health profile.

To achieve this, we designed and tested two distinct prompt templates:

Dietary Advice Template – focused on providing personalized nutritional reasoning and general guidance based on the user's health indicators.

Meal & Physical Plan Template – aimed at delivering a detailed 1-day meal plan along with a suggested physical activity routine.

These templates were applied after generating a diet recommendation using our predictive model, and we compare between them   in the section below.

In [1]:
%pip install pandas scikit-learn ipywidgets
%pip install openai
%pip install python-dotenv


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import necessary libraries
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split  # Splits data into training and testing sets for model evaluation
from sklearn.ensemble import RandomForestClassifier  # Implements the Random Forest algorithm for classification tasks
from sklearn.svm import SVC # Implements Support Vector Classification (SVM), a powerful machine learning algorithm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # Provides plotting functionality for data visualization 
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix 
import os 
from dotenv import load_dotenv 
from openai import OpenAI  
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from IPython.display import display, HTML
import ipywidgets as widgets
import pandas as pd


# Import evaluation metrics:
# - accuracy_score: Measures the percentage of correctly classified samples.
# - classification_report: Provides precision, recall, F1-score, and support for each class.
# - confusion_matrix: Displays the number of correct and incorrect predictions for each class.


In [3]:
# Load the API key from the .env file 
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
# Initialize OpenAI client 
client = OpenAI(api_key=api_key) 



In [4]:
# Load the preproccessting dataset
csv_path = "dataset_preprocessed (3).csv"  
df = pd.read_csv(csv_path)

# Define feature columns and target column
feature_cols = ['Gender', 'Disease_Type', 'Physical_Activity_Level', 'Age_Range', 'BMI_Range', 'Calorie_Range']
target_col = 'Diet_Recommendation'

# Select features and target
X = df[feature_cols]  
y = df[target_col]  


# Split the dataset into training (80%) and testing (20%) sets.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2,random_state=1, stratify=y
)



In [ ]:
categorical_features = ['Gender', 'Disease_Type', 'Physical_Activity_Level', 'Age_Range', 'BMI_Range', 'Calorie_Range']

preprocessor = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

svm_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', SVC(
        kernel='rbf',
        C=0.07,
        gamma='auto',
        random_state=42
    ))
])

svm_pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Gender', 'Disease_Type',
                                                   'Physical_Activity_Level',
                                                   'Age_Range', 'BMI_Range',
                                                   'Calorie_Range'])])),
                ('classifier', SVC(C=0.07, gamma='auto', random_state=42))])

In [ ]:
# Map for model output
diet_map = {0: 'Balanced', 1: 'Low Carb', 2: 'Low Sodium'}

# Feature columns
feature_cols = ['Gender', 'Disease_Type', 'Physical_Activity_Level', 'Age_Range', 'BMI_Range', 'Calorie_Range']

# Encoding for the model
label_encodings = {
    'Gender': {'Female': 0, 'Male': 1},
    'Disease_Type': {'None': 3, 'Diabetes': 0, 'Hypertension': 1, 'Obesity': 2},
    'Physical_Activity_Level': {'Sedentary': 2, 'Moderate': 1, 'Active': 0},
    'Age_Range': {'10-20': 0, '20-30': 1, '30-40': 2, '40-50': 3, '50-60': 4, '60-70': 5, '70-80': 6},
    'BMI_Range': {'10-20': 0, '20-30': 1, '30-40': 2, '40-50': 3},
    'Calorie_Range': {'1000-2000': 0, '2000-3000': 1, '3000-4000': 2}
}

# Dropdown widgets
dropdowns = {
    'Template': widgets.Dropdown(options=["dietary_advice", "meal_physical_plan"], description="Template:", value="dietary_advice"),
    'Gender': widgets.Dropdown(options=['Female', 'Male'], description='Gender'),
    'Disease_Type': widgets.Dropdown(options=['None', 'Diabetes', 'Hypertension', 'Obesity'], description='Disease'),
    'Physical_Activity_Level': widgets.Dropdown(options=['Sedentary', 'Moderate', 'Active'], description='Activity'),
    'Age_Range': widgets.Dropdown(options=['10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80'], description='Age'),
    'BMI_Range': widgets.Dropdown(options=['10-20', '20-30', '30-40', '40-50'], description='BMI'),
    'Calorie_Range': widgets.Dropdown(options=['1000-2000', '2000-3000', '3000-4000'], description='Calories'),
}

submit_button = widgets.Button(description="Submit")
output = widgets.Output()

# GPT explanation function
def gpt_explain_diet(symptom_features, predicted_class, template_type):
    prompt = (
        f"A user has the following profile:\n"
        f"- Gender: {symptom_features['Gender']}\n"
        f"- Disease Type: {symptom_features['Disease_Type']}\n"
        f"- Physical Activity Level: {symptom_features['Physical_Activity_Level']}\n"
        f"- Age Range: {symptom_features['Age_Range']}\n"
        f"- BMI Range: {symptom_features['BMI_Range']}\n"
        f"- Calorie Intake Range: {symptom_features['Calorie_Range']}\n\n"
        f"The predicted diet recommendation is plan #{predicted_class} ({diet_map[predicted_class]}).\n"
    )
    if template_type == "dietary advice":
        prompt += f"Please explain why this plan is appropriate and give helpful dietary advice."
    elif template_type == "meal_physical_plan9o":
        prompt += f"Please Provide a detailed 1-day meal plan and physical activity plan for this user."
    
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a nutritionist assistant providing helpful advice."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=400
        )
        return response.choices[0].message.content
    except Exception as e:
        return f" GPT Error: {e}"

# Submit button logic
def on_submit(change):
    # Get the selected template
    selected_template = dropdowns['Template'].value
    
    # Get user input (excluding the template dropdown)
    user_input = {col: dropdowns[col].value for col in feature_cols}  # readable
    encoded_input = {col: label_encodings[col][val] for col, val in user_input.items()}  # encoded
    encoded_df = pd.DataFrame([encoded_input])

    prediction = svm_pipeline.predict(encoded_df)[0]  # Model prediction

    # Get GPT advice based on readable user input and selected template
    gpt_response = gpt_explain_diet(user_input, prediction, selected_template)

    output.clear_output()
    with output:
        display(HTML(f"<h3 style='color: green;'>Predicted Diet Recommendation: {diet_map[prediction]}</h3>"))
        display(HTML(f"<p><strong>GPT Advice ({selected_template}):</strong><br>{gpt_response}</p>"))

# Button action
submit_button.on_click(on_submit)

# Display UI
print("Fill in your details and click Submit:")
display(*(dropdowns.values()), submit_button, output)

🔽 Fill in your details and click Submit:


Dropdown(description='Template:', options=('dietary_advice', 'meal_physical_plan'), value='dietary_advice')

Dropdown(description='Gender', options=('Female', 'Male'), value='Female')

Dropdown(description='Disease', options=('None', 'Diabetes', 'Hypertension', 'Obesity'), value='None')

Dropdown(description='Activity', options=('Sedentary', 'Moderate', 'Active'), value='Sedentary')

Dropdown(description='Age', options=('10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80'), value='10…

Dropdown(description='BMI', options=('10-20', '20-30', '30-40', '40-50'), value='10-20')

Dropdown(description='Calories', options=('1000-2000', '2000-3000', '3000-4000'), value='1000-2000')

Button(description='Submit', style=ButtonStyle())

Output()

In [13]:
# Data table with justifications based on actual outputs
data = {
    "Criteria": [
        "Text of Prompt",
        "Tone",
        "Clarity of Structure",
        "Level of Detail",
        "AI Guidance Strength"
    ],
    "Template 1: dietary_advice": [
        "Prompt asks GPT to explain why the prediction is suitable and give general dietary advice. It leads to an informative, advisory output with flexible content.",
        "The tone is friendly and supportive — it speaks directly to the user with empathy and general tips.",
        "Structure flows like an article: 7 numbered points but without strict segmentation for meal/activity types.",
        "Covers general nutritional advice like avoiding sugar and staying hydrated, but no specific meals or schedules.",
        "Because the prompt is open-ended, GPT produces varied results depending on context — flexible but less controlled."
    ],
    "Template 2: meal_physical_plan": [
        "Prompt is specific — it asks for a 1-day plan including meals and activities, pushing GPT to output structured content.",
        "The tone is professional and direct — it sounds like a written plan by a clinical nutritionist.",
        "Very structured: clear enumeration (1–6) with distinct focus areas like breakfast/lunch/portion sizes/monitoring.",
        "Gives specific food types (e.g., poultry, tofu, legumes), activity types, hydration goals, and diabetes-specific actions.",
        "Prompt gives GPT strong direction — outputs are consistent, clearly segmented, and aligned with real health planning."
    ]
}
# Set display option to show full column content
pd.set_option('display.max_colwidth', None)


# Display as pandas DataFrame
df_output_based = pd.DataFrame(data)
df_output_based  # This will still display the table normally, just without styling



,Criteria,Template 1: dietary_advice,Template 2: meal_physical_plan
0,Text of Prompt,"Prompt asks GPT to explain why the prediction is suitable and give general dietary advice. It leads to an informative, advisory output with flexible content.","Prompt is specific — it asks for a 1-day plan including meals and activities, pushing GPT to output structured content."
1,Tone,The tone is friendly and supportive — it speaks directly to the user with empathy and general tips.,The tone is professional and direct — it sounds like a written plan by a clinical nutritionist.
2,Clarity of Structure,Structure flows like an article: 7 numbered points but without strict segmentation for meal/activity types.,Very structured: clear enumeration (1–6) with distinct focus areas like breakfast/lunch/portion sizes/monitoring.
3,Level of Detail,"Covers general nutritional advice like avoiding sugar and staying hydrated, but no specific meals or schedules.","Gives specific food types (e.g., poultry, tofu, legumes), activity types, hydration goals, and diabetes-specific actions."
4,AI Guidance Strength,"Because the prompt is open-ended, GPT produces varied results depending on context — flexible but less controlled.","Prompt gives GPT strong direction — outputs are consistent, clearly segmented, and aligned with real health planning."


We implemented two distinct templates:
- `dietary_advice`
- `meal_physical_plan`

Each template generates a different style of output depending on the same input data. This section highlights their differences and the rationale behind including both.

---

###  Template Purpose Overview

| Template Name       | Purpose                                                                 |
|---------------------|-------------------------------------------------------------------------|
| `dietary_advice`    | Provides general dietary guidelines tailored to the user's condition.   |
| `meal_physical_plan`| Delivers a detailed, personalized daily meal and physical activity plan.|

---


###  Justification for Including Both Templates

Using both templates enhances the **usability and flexibility** of our system:

- `dietary_advice` is ideal for users who are at the awareness or exploration stage of managing their condition. It educates and motivates behavior change through general health advice.
  
- `meal_physical_plan` is better suited for users seeking specific, actionable plans they can apply directly. It transforms abstract predictions into concrete guidance.

Together, these templates ensure the system can support a wide range of users, from those new to dietary planning to those needing daily structure.